## Pearson and Spearman correlations between the top-5 explanations from RPS-LJE and Influence Functions
Table 7 (appendix) and Table 8 (appendix)

In [1]:
import numpy as np
import torch
path = '../saved_models/base'
file = np.load('{}/model/saved_outputs.npz'.format(path))
intermediate_train = torch.from_numpy(file['intermediate_train']).squeeze()
intermediate_test = torch.from_numpy(file['intermediate_test']).squeeze()
labels_train = file['labels_train']
labels_test = file['labels_test']
pred_train = file['pred_train'].squeeze()
pred_test = file['pred_test'].squeeze()

In [2]:
weight_matrix_influence = np.load('{}/calculated_weights/influence_weight_matrix.npz'.format(path), allow_pickle=True)['weight_matrix'].squeeze()
jaccobian_test = np.load('{}/calculated_weights/influence_weight_matrix.npz'.format(path), allow_pickle=True)['jaccobian_test']
weight_matrix_ours = np.load('{}/calculated_weights/ours_weight_matrix_with_lr_1e-05.npz'.format(path), allow_pickle=True)['weight_matrix'].squeeze()


In [3]:

def get_influence_order(test_point=None):
    tmp = jaccobian_test[test_point, 0]@ np.transpose(weight_matrix_influence)
    pos_idx = np.argsort(tmp, axis=0)
    return pos_idx, tmp

def get_ours_order(test_point=None):
    true_class = labels_test[test_point]
    tmp = np.dot(weight_matrix_ours,
                 intermediate_test[test_point,:])
    if true_class == 1:
        pos_idx = np.flip(np.argsort(tmp), axis=0)
        tmp = -tmp
    else:
        pos_idx = np.argsort(tmp)
    return pos_idx,tmp

In [4]:
from scipy.stats import spearmanr, pearsonr

pearson_correlation_list = []
spearman_correlation_list = []
kendall_correlation_list= []
for i in range(len(intermediate_test)):
    order_ours, importance_ours = get_ours_order(test_point=i)
    order_inf, importance_inf = get_influence_order(test_point=i)

    top_5_idx = list(set(order_inf[:5]).union(set(order_ours[:5])))

    pearson_correlation_list.append(pearsonr(importance_inf[top_5_idx],
                                             importance_ours[top_5_idx])[0])
    spearman_correlation_list.append(spearmanr(importance_inf[top_5_idx],
                                               importance_ours[top_5_idx])[0])

In [5]:
np.quantile(pearson_correlation_list, q=[0.0001,0.001,0.01,0.1,0.3,0.5,0.7,0.9],
            interpolation='lower')

array([0.14841145, 0.77886628, 0.95136279, 0.99556548, 0.99920157,
       0.99973452, 0.99991155, 0.9999829 ])

In [6]:
np.quantile(spearman_correlation_list, q=[0.0001,0.001,0.01,0.1,0.3,0.5,0.7,0.9],
            interpolation='lower')


array([0.1, 0.4, 0.7, 0.9, 1. , 1. , 1. , 1. ])